# Set Up Databases and Populating Tickers and Historical Stock Data

# 1. Setup & Imports

In [ ]:
import sys
import os
import sqlite3
from pathlib import Path
from datetime import datetime, timedelta

# Define BASE_DIR and adjust sys.path before importing
BASE_DIR = Path(os.getcwd()).parent  # Points to stat_656_autotrader/ from Notebooks/
sys.path.append(str(BASE_DIR))
print(f"Project root added to sys.path: {BASE_DIR}")

# Import paths from the config module
from src.config import BASE_DIR, DB_DIR

from src.db_schema import DATABASES
from src.etl.populate_tickers import recreate_database, populate_tickers
from src.etl.populate_prices import ensure_prices_table, populate_prices
from src.utils.db_utils import fetch_all_asset_metadata, fetch_all_asset_prices
from src.utils.alpaca_utils import populate_alpaca_full_price_history

Project root added to sys.path: d:\dev\stat_656_autotrader


# 2. Initialize SQLite Databases

In [2]:
DB_DIR.mkdir(exist_ok=True)

def setup_databases():
    """Initialize databases and tables based on defined schemas."""
    for db_name, schemas in DATABASES.items():
        db_path = DB_DIR / db_name

        try:
            with sqlite3.connect(db_path) as conn:
                conn.execute("PRAGMA foreign_keys = ON;")
                cursor = conn.cursor()
                for schema in schemas:
                    cursor.execute(schema)
                conn.commit()
                print(f"✅ Tables created/verified in '{db_name}'")
        except sqlite3.Error as e:
            print(f"❌ Error setting up '{db_name}': {e}")

# Run database setup
setup_databases()

✅ Tables created/verified in 'assets.db'
✅ Tables created/verified in 'portfolio_management.db'
✅ Tables created/verified in 'accounting.db'
✅ Tables created/verified in 'modeling.db'
✅ Tables created/verified in 'exogenous.db'
✅ Tables created/verified in 'db_change_log.db'


# 3. Populate Asset Metadata (Tickers)

In [3]:
print("\n🔄 Recreating and populating tickers database...")
recreate_database()
populate_tickers()
print("✅ Tickers populated successfully.")


🔄 Recreating and populating tickers database...
✅ Tickers populated successfully.


In [4]:
# View the asset metadata
asset_metadata_df = fetch_all_asset_metadata()
print("DataFrame shape:", asset_metadata_df.shape)
print("DataFrame columns:", asset_metadata_df.columns.tolist())
display(asset_metadata_df.head(20))

Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db
Raw rows fetched: [(1, 'GLDG', 'GoldMining Inc.', 'AMEX', 'us_equity', 1, '2025-04-14', None, '2025-04-14 14:15:29'), (2, 'TEAF', 'Tortoise Sustainable and Social Impact Term Fund', 'NYSE', 'us_equity', 1, '2025-04-14', None, '2025-04-14 14:15:29'), (3, 'TOROV', 'Toro Corp. Common Stock Ex-distribution When Issued', 'NASDAQ', 'us_equity', 1, '2025-04-14', None, '2025-04-14 14:15:29'), (4, 'LRGE', 'Legg Mason ETF Investment Trust ClearBridge Large Cap Growth ESG ETF', 'NASDAQ', 'us_equity', 1, '2025-04-14', None, '2025-04-14 14:15:29'), (5, 'GLPG', 'Galapagos NV American Depositary Shares', 'NASDAQ', 'us_equity', 1, '2025-04-14', None, '2025-04-14 14:15:29')]
Column names from DB: ['asset_id', 'symbol', 'name', 'exchange', 'asset_type', 'is_active', 'date_added', 'date_removed', 'fetched_at']
DataFrame shape: (7471, 9)
DataFrame columns: ['asset_id', 'symbol', 'name', 'exchange', 'asset_type', 'is_active', 'date_adde

,asset_id,symbol,name,exchange,asset_type,is_active,date_added,date_removed,fetched_at
0,1,GLDG,GoldMining Inc.,AMEX,us_equity,1,2025-04-14,None,2025-04-14 14:15:29
1,2,TEAF,Tortoise Sustainable and Social Impact Term Fund,NYSE,us_equity,1,2025-04-14,None,2025-04-14 14:15:29
2,3,TOROV,Toro Corp. Common Stock Ex-distribution When I...,NASDAQ,us_equity,1,2025-04-14,None,2025-04-14 14:15:29
3,4,LRGE,Legg Mason ETF Investment Trust ClearBridge La...,NASDAQ,us_equity,1,2025-04-14,None,2025-04-14 14:15:29
4,5,GLPG,Galapagos NV American Depositary Shares,NASDAQ,us_equity,1,2025-04-14,None,2025-04-14 14:15:29
5,6,FUND,"Sprott Focus Trust, Inc. Common Stock",NASDAQ,us_equity,1,2025-04-14,None,2025-04-14 14:15:29
6,7,FER,Ferrovial SE Ordinary Shares,NASDAQ,us_equity,1,2025-04-14,None,2025-04-14 14:15:29
7,8,MYI,"BLACKROCK MUNIYIELD QUALITY FUND III, INC.",NYSE,us_equity,1,2025-04-14,None,2025-04-14 14:15:29
8,9,XFIX,"The RBB Fund, Inc. F/m Opportunistic Income ETF",NASDAQ,us_equity,1,2025-04-14,None,2025-04-14 14:15:29
9,10,XELB,"Xcel Brands, Inc. Common Stock",NASDAQ,us_equity,1,2025-04-14,None,2025-04-14 14:15:29


In [5]:
asset_metadata_df['exchange'].unique()

array(['AMEX', 'NYSE', 'NASDAQ'], dtype=object)

# 4. Populate Historical Stock Prices

In [ ]:
print("\n🔄 Ensuring prices table exists and populating historical prices...")
#ensure_prices_table()
populate_alpaca_full_price_history()
print("✅ Historical stock prices populated successfully.")


🔄 Ensuring prices table exists and populating historical prices...
Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db


Alpaca Download Progress: 100%|██████████| 7471/7471 [1:54:59<00:00,  1.08it/s]  



Fetched 10084360 rows of stock data in 6899.86 seconds.
Processed 7471 tickers, missing 2869 tickers, 2869 tickers did not have enough time.
Processing time per ticker: 0.92 seconds.


Inserting Prices into DB: 100%|██████████| 10084360/10084360 [15:27<00:00, 10878.34it/s]


✅ Historical stock prices populated successfully.


In [7]:
# Fetch and display the historical prices
print("\n📊 Fetching and displaying historical stock prices...")
prices_df = fetch_all_asset_prices()

# Display basic info and the first 20 rows
print(f"DataFrame shape: {prices_df.shape}")
print(f"Columns: {prices_df.columns.tolist()}")
display(prices_df.head(20))


📊 Fetching and displaying historical stock prices...
Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db
DataFrame shape: (10084360, 11)
Columns: ['price_id', 'asset_id', 'symbol', 'date', 'open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'fetched_at']


,price_id,asset_id,symbol,date,open,high,low,close,adjusted_close,volume,fetched_at
0,3504,5,GLPG,2016-01-04,61.52,61.6900,59.7100,60.9600,None,None,2025-04-14 16:11:27
1,5837,6,FUND,2016-01-04,5.71,5.7100,5.6220,5.6600,None,None,2025-04-14 16:11:27
2,8170,8,MYI,2016-01-04,14.74,14.8000,14.6100,14.7900,None,None,2025-04-14 16:11:28
3,12835,11,VGM,2016-01-04,13.28,13.4200,13.2800,13.4200,None,None,2025-04-14 16:11:28
4,15168,12,VGI,2016-01-04,13.94,14.1200,13.9100,14.0700,None,None,2025-04-14 16:11:28
5,17501,13,SMP,2016-01-04,36.75,37.6400,36.0300,37.0100,None,None,2025-04-14 16:11:28
6,19834,16,TAX,2016-01-04,23.11,23.8500,23.0200,23.1500,None,None,2025-04-14 16:11:29
7,22167,21,TARA,2016-01-04,15.00,15.0000,13.3800,14.0400,None,None,2025-04-14 16:11:29
8,24500,22,TAOP,2016-01-04,1.56,1.7800,1.4100,1.7200,None,None,2025-04-14 16:11:29
9,26833,23,TANH,2016-01-04,5.15,5.4700,5.1200,5.4700,None,None,2025-04-14 16:11:29
